- author: Lee Meng
- date: 2019-03-25 09:00
- title: 讓 AI 寫點金庸：如何用 TensorFlow 2.0 及 TensorFlow.js 寫天龍八部
- slug: how-to-generate-interesting-text-with-tensorflow2-and-tensorflow-js
- tags: TensorFlow, TensorFlow.js, 自然語言處理
- description: TODO
- summary: TODO
- image: text-generation-cover.jpg
- image_credit_url: 
- status: draft

<link rel="stylesheet" href="{static}tfjs-apps/lstm-text-generation/index.css" />

<blockquote style="margin-bottom: 1rem">
    <p>
        木婉清轉頭向他，背脊向著南海鱷神，低聲道：「你是世上第一個見到我容貌的男子！」緩緩拉開了面幕。段譽登時全身一震，眼前所見，如新月清暉，如花樹堆雪，一張臉秀麗絕俗。
        <br>
        <span style="float:right;margin-right: 1.5rem">第四回：崖高人遠</span>
        <br>
    </p>
</blockquote>

<br/>

[《天龍八部》](https://bit.ly/2TUycBQ)一直是我最喜歡的[金庸著作](https://zh.wikipedia.org/wiki/%E9%87%91%E5%BA%B8%E4%BD%9C%E5%93%81)之一，最近重新翻閱，有很多新的感受。

閱讀到一半我突發奇想，決定用[深度學習](https://leemeng.tw/deep-learning-resources.html)來訓練一個能夠生成《天龍八部》的[循環神經網路](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E6%9C%89%E8%A8%98%E6%86%B6%E7%9A%84%E5%BE%AA%E7%92%B0%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF)。訓練結果還不完美，但我認為已經很有娛樂性質，且有時已經能夠產生令人驚嘆或是捧腹大笑的文章了。

因此我決定使用 [Tensorflow.js](https://www.tensorflow.org/js) 將利用 [TensorFlow 2.0](https://www.tensorflow.org/alpha) 訓練出來的模型弄上線，讓你也能實際看看這個 AI 上嗑了什麼藥。

!image
- lstm-text-generation/dali-old-castle.jpg
- 大理古城一隅，段譽出身之地

在 demo 之後，針對那些熟悉 [Python](https://www.python.org/) 或是 [JavaScript](https://www.javascript.com/) 的你，我將簡單地說明如何使用 TensorFlow 以及 Tensorflow.js 來實作這樣的 AI 應用。

希望你能從中得到些啟發，並在閱讀本文後實際動手做點自己的東西。

## Demo：生成全新的天龍八部橋段

本篇使用一個十分簡單的[長短期記憶 RNN](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E8%A8%98%E6%86%B6%E5%8A%9B%E5%A5%BD%E7%9A%84-LSTM-%E7%B4%B0%E8%83%9E) 來生成文章。在多次「閱讀」天龍八部之後，這個模型可以在給定一段新文本的情況下，逐字產生類似風格的文章。

比方說給定書中的一個橋段：

```text
烏老大偏生要考一考慕容復，說道：「慕容公子，你瞧這不是大大的
```

模型順著上面的話，接著產生：

```text
不算？」馬夫人道：「不錯，咱們非要尋死不可。」

段譽大喜，說道：「小姑娘，你待我這麼好，鬼鬼祟祟，一切又不聽你的話，你管甚麼老兄弟不相干，我去幫過彥之。」

王夫人哼了一聲，說道：「這裏是甚麼話？」段譽道：「不行！你別過來。用真蠻子，我便將這件事了，一大惡人擠在地下，立時便會斃命，那便如何是好？」
```

模型產生的文章內容本身很ㄎ一ㄤ，惹人發笑，但用詞本身很天龍八部。（至少我自己寫不出這樣的內容）

!image
- lstm-text-generation/antony-xia-522590-unsplash.jpg
- 姑蘇慕容家所在的蘇州

現在馬上就讓我們產生一些新的橋段吧！首先將已經訓練好的模型載入你的瀏覽器。依據你的網路速度，載入時間會有所差異。

成功載入後只要不重新整理此頁面，你將可以不斷地使用同個模型產生新的文章。

<section style="margin-bottom: 3rem">
    <button id="load-model" style="display:inline-block">載入模型</button>
    <div id="app-status" style="display:inline-block"></div>
</section>

另外你會發現有 2 個可供你調整的參數：

<section style="margin-bottom: 3rem">
    <div>
        <span class="input-title">生成長度（字單位）</span>
        <input id="generate-length" value="150"></input>
    </div>
    <div>
        <span class="input-title">生成溫度（隨機度）</span>
        <input id="temperature" value="0.6"></input>
    </div>
</section>

第一次可以直接使用預設值。現在點擊「生成文章」來產生全新的天龍八部橋段：

<section style="margin-bottom: 3rem">
    <div>
        <button id="generate-text" disabled="true">生成文章</button>
        <button id="initialize-seed" disabled="true">重置輸入</button>
    </div>
</section>
<section style="margin-bottom: 3rem">
    <div>
        <span class="input-title">起始句子：</span>
        <span id="text-generation-status" style="display: none"></span>
        <textarea id="seed-text" value="" rows="1" style="min-height: 6em">蕭峯吃了一驚，心想：「哥哥大喜之餘，說話有些忘形了，眼下亂成</textarea>
    </div>
</section>
<section style="margin-bottom: 3rem">
    <div>
        <span class="input-title">生成結果：</span>
        <textarea id="generated-text" readonly="true" value="" rows="10"></textarea>
    </div>
</section>

如何？希望模型產生的結果有令你會心一笑。它當初可快把我逗死了。

現在你可以嘗試幾件事情：
- 點**生成文章**來讓模型依據同輸入產生新橋段
- 點**重置輸入**來隨機取得一個新的起始句子
- 增加模型生成的**文章長度**
- 調整**生成溫度**來改變文章的變化性

!image
- lstm-text-generation/chris-rhoads-254898-unsplash.jpg

生成溫度是一個介於 0 到 1 的值，而當溫度越高，模型產生出來的結果越隨機、越不可預測（也就越ㄎㄧㄤ）；而溫度越低，產生的結果就會越像天龍八部原文。優點是真實，但同時字詞的重複性也會提升。

!quote
- 機器並沒有情感，只有人類可以賦予事物意義。我們無法讓機器自動找出所謂的最佳生成溫度，因為人的感覺十分主觀：找出你自己覺得最適合的溫度來生成文章。

如果你沒有打算深入探討技術細節，那只需要記得在這篇文章裡頭的模型是一個以「字」為單位的語言模型（Character-based Language Model）即可：給定一連串已經出現過的字詞，模型會想辦法去預測出下一個可能出現的字。

!image
- lstm-text-generation/raychan-1229841-unsplash.jpg

值得注意的是，我們並不單純是拿出現機率最高的字出來當結果，這樣太無趣了。

每次機器做預測前都會拿著一個包含大量中文字的機率分布 p，在決定要吐出哪個字時，會對該機率分佈 p 做抽樣，從中隨機選出一個字。因此就跟你在上面 demo 看到的一樣，就算輸入句子相同，每次模型仍然可能生成完全不同的文章。

!image
- lstm-text-generation/max-felner-448887-unsplash.jpg
- 抽樣的過程類似擲骰子，儘管有些結果較易出現，你還是有機會骰到豹子

因為隨機抽樣的關係，每次模型產生的結果基本上都是獨一無二的。

如果你在生成文章的過程中得到什麼有趣的虛擬橋段，都歡迎留言或是透過 SNS 與我分享！

## 模型是怎麼被訓練的

在看完 demo 以後，你可能會好奇這個模型是怎麼被訓練出來的。

實際的開發流程大致可以分為兩個部分：
- [用 TensorFlow 2.0 訓練一個 LSTM 模型](https://www.tensorflow.org/alpha/tutorials/sequences/text_generation)
- [使用 TensorFlow.js 部屬該模型](https://github.com/tensorflow/tfjs-examples/tree/master/lstm-text-generation)

這些在 TensorFlow 以及 TensorFlow.js 官網都有十分詳細的說明。

!image
- lstm-text-generation/tf-demo.png
- 這篇文章參考了不少 TensorFlow 官網（左）及 TensorFlow.js 線上 demo（右）的程式碼

如果你也想開發一個類似的應用，閱讀官方教學中你所熟悉的語言版本（Python / JavaScript）是最直接的作法：
- [TensorFlow 2.0 Alpha - Text generation with an RNN](https://www.tensorflow.org/alpha/tutorials/sequences/text_generation)
- [TensorFlow.js Example: Train LSTM to Generate Text](https://github.com/tensorflow/tfjs-examples/tree/master/lstm-text-generation)

因為官方已經有提供能在 [Google Colab](https://colab.research.google.com/) 上使用 GPU [訓練 LSTM 的教學筆記本](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/sequences/text_generation.ipynb)，本文便不再另行提供。

!image
- lstm-text-generation/simon-abrams-286276-unsplash.jpg

另外，符合以下條件可以讓你更輕鬆地閱讀接下來的內容：
- 熟悉 [Python](https://www.python.org/)
- 碰過 [Keras](https://keras.io/) 或是 [TensorFlow](https://www.tensorflow.org/)
- 具備[機器學習 & 深度學習基礎](https://leemeng.tw/deep-learning-resources.html#courses)
- 了解何謂[循環神經網路](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E6%9C%89%E8%A8%98%E6%86%B6%E7%9A%84%E5%BE%AA%E7%92%B0%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF)以及[長短期記憶](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E8%A8%98%E6%86%B6%E5%8A%9B%E5%A5%BD%E7%9A%84-LSTM-%E7%B4%B0%E8%83%9E)

如果你是喜歡先把基礎打好的人，可以先查閱我上面附的這些資源連結。

## TensorFlow 2.0 開發

平常有在接觸深度學習的讀者或許都已經知道，最近 TensorFlow 隆重推出 [2.0 Alpha 預覽版](https://www.tensorflow.org/alpha)，希望透過全新的 API 讓更多人可以輕鬆地開發機器學習以及深度學習應用。

當初撰寫本文的其中一個目的，也是想趁著這次大改版來讓自己熟悉 TensorFlow 2.0 的開發方式。

<div class="resp-container">
    <iframe class="resp-iframe" src="https://www.youtube.com/embed/TTQQiJ-mHYA" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>

對我來說，TensorFlow 2.0 有幾個重點：
- 將 [tf.keras](https://www.tensorflow.org/alpha/guide/keras/overview) 視為其官方高級 API
- 方便除錯的 [Eager Execution](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/eager.ipynb) 成為預設值
- 負責讀取、處理大量數據的 [tf.data](https://www.tensorflow.org/alpha/guide/data_performance) API
- 自動幫你建構計算圖的 [tf.function](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/function)

在這篇文章裡頭會看到前 3 者。下節列出的程式碼皆在 [Google Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/sequences/text_generation.ipynb) 上用最新版本的 TensorFlow 2.0 Nightly 執行。

```bash
pip install tf-nightly-gpu-2.0-preview
```

如果有 GPU 則強烈建議安裝 GPU 版本的 TF Nightly，訓練速度跟 CPU 版本可以差到 10 倍以上。

## 深度學習專案步驟

好戲終於登場。

如同多數的深度學習專案，要訓練一個以 LSTM 為基礎的語言模型，你大致需要走過以下幾個步驟：

!image
- lstm-text-generation/deep-learning-pj-steps-menglee.jpg
- 開發一個 DL 專案時我常用的流程架構

這個流程是一個大方向，依據不同情境你可能需要自行調整流程來符合自己的需求。

這篇文章會用 TensorFlow 2.0 簡單地帶你走過所有步驟。

### 1. 定義問題及要解決的任務

很明顯地，在訓練模型前首先得確認我們的問題（Problem）以及想要交給機器解決的任務（Task）是什麼。

前面已經提過，我們的目標就是要找出一個天龍八部的語言模型（Language Model），讓該模型在被餵進一段文字以後，能吐出類似天龍八部的文章。

<div class="resp-container">
    <iframe class="resp-iframe" src="https://www.youtube.com/embed/f1KUUz7v8g4?list=PLJV_el3uVTsPMxPbjeX7PicgWbY7F8wW9" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>
<center>
    十分推薦李宏毅教授講解序列生成的影片
    <br>
    <br>
</center>

這實際上是一個[序列生成（Sequence Generation）](https://youtu.be/f1KUUz7v8g4?list=PLJV_el3uVTsPMxPbjeX7PicgWbY7F8wW9)問題，而機器所要解決的任務也變得明確：給定一段文字單位的序列，它要能吐出下一個合理的文字單位。

這邊說的文字單位（Token）可以是
- 字（Character，如劍、寺、雲）
- 詞（Word，如吐蕃、師弟、阿修羅）

本文則使用「字」作為一個文字單位。假設有一個天龍八部的句子：

```text
『六脈神劍經』乃本寺鎮寺之寶，大理段氏武學的至高法要。
``` 

這時候裡頭的每個字包含標點符號都是一個文字單位，整個句子就構成一個文字序列。我們可以擷取一部份的句子：

```text
『六脈神劍經』乃本寺鎮寺之寶，大理段氏武
```

接著在訓練模型時要求它讀入這段文字，並預測出原文裡頭下一個出現的字：`學`。

一旦訓練完成，就能得到你開頭看到的那個以字為單位的語言模型了。

### 2. 準備原始數據、資料清理

巧婦難為無米之炊，沒有數據一切免談。

In [1]:
#ignore
import tensorflow as tf
import os
tf.enable_eager_execution()
with open("../../raw_text.txt", "r") as f:
    text = f.read()
    num_words = len(set(text))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

!image
- lstm-text-generation/caroline-attwood-243834-unsplash.jpg

我在網路上自行蒐集了天龍八部原文，做些簡單的數據清理後發現整本小說總共約含 120 萬個中文字，實在是一部曠世巨作。儘管因為版權問題不宜提供下載連結，Google 是你的好朋友。

現在假設我們把原文全部存在一個 Python 字串 `text` 裡頭，則部分內容可能如下：

In [4]:
# 隨意取出第 9505 到 9702 的中文字
print(text[9505:9702])

咱們見敵方人多，不得師父號令，沒敢隨便動手。」左子穆道：「嗯，來了多少人？」干光豪道：「大約七八十人。」左子穆嘿嘿冷笑，道：「七八十人，便想誅滅無量劍了？只怕也沒這麼容易。」

龔光傑道：「他們用箭射過來一封信，封皮上寫得好生無禮。」說著將信呈上。

左子穆見信封上寫著：「字諭左子穆」五個大字，便不接信，說道：「你拆來瞧瞧。」龔光傑道：「是！」拆開信封，抽出信箋。

那少女在段譽耳邊低聲道：


我們也可以看看整本小說裡頭包含多少中文字：

In [2]:
num_words = len(set(text))
print("天龍八部小說共有 {} 中文字".format(
    len(text)))
print("包含了 {} 個獨一無二的字".format(
    num_words))

天龍八部小說共有 1235431 中文字
包含了 4330 個獨一無二的字


相較於英文只有 26 個字母，中文字自然是比較多的。

如同我們在[寫給所有人的自然語言處理與深度學習入門指南](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html)看過的，要將文本數據丟入神經網路，我們得先做些前處理，將這些中文字對應到一個個的數字（或稱索引）才行。

我們可以使用 `tf.keras` 裡頭的 `Tokenizer` 幫我們把整篇小說建立字典，並將同樣的中文字對應到同樣的索引數字：

In [3]:
import tensorflow as tf

# 初始化一個以字為單位的 Tokenizer
tokenizer = tf.keras\
    .preprocessing\
    .text\
    .Tokenizer(
        num_words=num_words,
        char_level=True,
        filters=''
)

# 讓 tokenizer 讀過天龍八部全文，
# 為每個出現的字建立字典並將中文字轉
# 成對應的數字索引
tokenizer.fit_on_texts(text)
text_as_int = tokenizer\
    .texts_to_sequences([text])[0]

# 隨機選取一個片段文本方便之後做說明
s_idx = 21004
e_idx = 21020
partial_indices = \
    text_as_int[s_idx:e_idx]
partial_texts = [
    tokenizer.index_word[idx] \
    for idx in partial_indices
]

# 渲染結果，可忽略
print("原本的中文字序列：")
print()
print(partial_texts)
print()
print("-" * 20)
print()
print("轉換後的索引序列：")
print()
print(partial_indices)

原本的中文字序列：

['司', '空', '玄', '雙', '掌', '飛', '舞', '，', '逼', '得', '牠', '無', '法', '近', '前', '。']

--------------------

轉換後的索引序列：

[557, 371, 215, 214, 135, 418, 1209, 1, 837, 25, 1751, 49, 147, 537, 111, 2]


很明顯地，現在整部天龍八部都已經被轉成一個巨大的數字序列，每一個數字代表著一個獨立的中文字。

我們可以換個方向再看一次：

In [128]:
#ignore
print("{0:10}{1:10}".format(
        "人類看的中文字", "機器看的輸入索引"))
print("-" * 30)
for idx in partial_indices:
    word = tokenizer.index_word[idx]
    print("{0:10}{1:10}".format(word, idx))

人類看的中文字   機器看的輸入索引  
------------------------------
司                557
空                371
玄                215
雙                214
掌                135
飛                418
舞               1209
，                  1
逼                837
得                 25
牠               1751
無                 49
法                147
近                537
前                111
。                  2


### 3. 建立能丟入模型的資料集

做完基本的數據前處理以後，我們需要將 `text_as_int` 這個巨大的數字序列轉換成神經網路容易消化的格式與大小。

In [132]:
print(
    "小說的中文字數：", 
    len(text_as_int), 
    "\n",
    "前 5 個索引：",
    text_as_int[:5]
)

小說的中文字數： 1235431 
 前 5 個索引： [1639, 148, 3, 3, 280]


!quote
- 在建立資料集時，你要先能想像最終交給模型的數據長什麼樣子。這樣能幫助你對數據做適當的轉換。

依照不同的機器學習任務，你會需要不同格式的資料形式。

在本文的序列生成任務裡頭，理想的模型要能依據前文來判斷出下一個中文字。因此我們要丟給模型的是一串代表某些中文字的數字序列：

In [133]:
print("實際丟給模型的數字序列：")
print(partial_indices)
print()
print("方便我們理解的文本序列：")
print(partial_texts)

實際丟給模型的數字序列：
[557, 371, 215, 214, 135, 418, 1209, 1, 837, 25, 1751, 49, 147, 537, 111, 2]

方便我們理解的文本序列：
['司', '空', '玄', '雙', '掌', '飛', '舞', '，', '逼', '得', '牠', '無', '法', '近', '前', '。']


而模型要給我們的理想輸出應該是向左位移一個字的結果：

In [135]:
print("實際丟給模型的數字序列：")
print(partial_indices[1:])
print()
print("方便我們理解的文本序列：")
print(partial_texts[1:])

實際丟給模型的數字序列：
[371, 215, 214, 135, 418, 1209, 1, 837, 25, 1751, 49, 147, 537, 111, 2]

方便我們理解的文本序列：
['空', '玄', '雙', '掌', '飛', '舞', '，', '逼', '得', '牠', '無', '法', '近', '前', '。']


為什麼是這樣的配對？

想一想，一個模型如果可以給我們這樣的輸出，代表它：
- 看到第一個輸入字 `司` 時可以正確輸出 `空`
- 在之前看過 `司`，且新輸入字為 `空` 的情況下，可以輸出 `玄` 
- 在之前看過 `司空`，且新輸入字為 `玄` 的情況下，可以輸出 `雙`
- 在之前看過 `司空玄雙掌飛`，且新輸入字為 `舞` 的情況下，可以輸出 `，`

當一個語言模型可以做到這樣的事情，就代表它已經掌握了**訓練文本**（此文中為天龍八部）裡頭用字的統計結構，因此我們可以用它來產生新的天龍八部文章。

你現在應該也可以了解，這個語言模型是專為天龍八部的文本所誕生的。畢竟日常生活中，給你 `舞` 這個字，你接 `，` 的機率有多少呢？

!image
- lstm-text-generation/niketh-vellanki-202943-unsplash.jpg

為了讓你加深印象，讓我把序列擺直，再次列出模型的輸入以及輸出關係：

In [29]:
#ignore
print("{0:4}{1:5}{2:7}{3:5}{4:6}".format(
        "時間點", "輸入字", 
        "輸入索引", "輸出字", 
        "輸出索引"))
print("-" * 37)
for t, idx in enumerate(partial_indices[:14]):
    if idx + 1 == len(partial_indices):
        break
    word = tokenizer.index_word[idx]
    next_idx = partial_indices[t + 1]
#     print(next_idx)
    next_word = tokenizer.index_word[next_idx]
    print("{0:4}{1:5}{2:9}{3:4}{4:7}".format(
        t + 1,
        ' '* 4 + word, 
        ' '* 5 + str(idx), 
        ' '* 5 + next_word, 
        ' '* 6 + str(next_idx)))

時間點 輸入字  輸入索引   輸出字  輸出索引  
-------------------------------------
   1    司     557      空      371
   2    空     371      玄      215
   3    玄     215      雙      214
   4    雙     214      掌      135
   5    掌     135      飛      418
   6    飛     418      舞      1209
   7    舞     1209     ，      1
   8    ，     1        逼      837
   9    逼     837      得      25
  10    得     25       牠      1751
  11    牠     1751     無      49
  12    無     49       法      147
  13    法     147      近      537
  14    近     537      前      111


每一列（row）是一個時間點，而
- **輸入索引**代表模型在當下時間吃進去的輸入
- **輸出索引**則代表我們要模型輸出的結果

輸入字・輸出字則只是方便我們理解對照，實際上模型只吃數字。

!image
- nlp-kaggle-intro/pop-zebra-754186-unsplash.jpg

現在我們了解一筆輸入・輸出該有的數據格式了。兩者皆是一個固定長度的數字序列，而後者是前者往左位移一個數字的結果。

但這只是一筆數據（以下說的一筆數據，都隱含了輸入序列以及對應的輸出序列的 2 個數字序列）。

在有 GPU 的情況下，我們常常會一次丟一批（batch）數據，讓 GPU 可以平行運算，加快訓練速度。

!image
- lstm-text-generation/gpu.jpg

現在假設我們想要一個資料集，而此資料集可以一次給我們 128 筆長度為 10 的輸入・輸出序列，則我們可以用 `tf.data` 這樣做：

In [149]:
# 方便說明，你可以設成更大的數字
# 讓模型從更長的序列預測結果
SEQ_LENGTH = 10  # 數字序列長度
BATCH_SIZE = 128 # 幾筆成對輸入/輸出

# text_as_int 是一個 python list
# 我們利用 from_tensor_slices 將其
# 轉變成 TensorFlow 最愛的 Tensor <3
characters = tf\
    .data\
    .Dataset\
    .from_tensor_slices(
        text_as_int)

# 將被以數字序列表示的天龍八部文本
# 拆成多個長度為 SEQ_LENGTH (10) 的序列
# 並將最後長度不滿 SEQ_LENGTH 的序列捨去
sequences = characters\
    .batch(SEQ_LENGTH + 1, 
           drop_remainder=True)

# 天龍八部全文所包含的成對輸入/輸出的數量
steps_per_epoch = \
    len(text_as_int) // SEQ_LENGTH

# 將一個片段的數字序列取頭作為輸入序列
# 並把向左位移一個字的數據序列作為輸出序列
def build_seq_pairs(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# 將每個片段序列分別取頭取尾，
# 建立輸入 / 輸出序列，
# 再將得到的所有數據隨機打亂順序
# 最後再一次拿出 BATCH_SIZE 筆數據作為
# 模型一次訓練步驟的所使用的資料
ds = sequences\
    .map(build_seq_pairs)\
    .shuffle(steps_per_epoch)\
    .batch(BATCH_SIZE, 
           drop_remainder=True)

這段程式碼不短，且初次接觸 `tf.data` 的讀者可能會覺得很陌生。

但內容不難，且在熟悉以後你可以用非常類似的方式呼叫 [TensorFlow Data API](https://www.tensorflow.org/guide/datasets) 來處理**任何**文本數據，不需要每次遇到新的文本都得重頭開始寫類似的功能（`batch`、`shuffle` etc）。

上面已經有非常多的註解幫助你理解發生了什麼事，但如果你想自己動手，可以參考[官方用 TensorFlow 2.0 訓練 LSTM 的 Colab 筆記本](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/sequences/text_generation.ipynb)。

雖然我不是酷拉皮卡，但如果要把上面用 `build_seq_pairs` 函式將一個文本序列拆成輸入・輸出序列的處理具現化的話，大概就像是下面這樣（假設序列長度為 6）：

```text
文本擷取的片段序列     輸入/輸出序列
----------------------------------
                 -> 烏老大拱手還
                 |
烏老大拱手還禮 -----
                 |
                 -> 老大拱手還禮


                 -> 星宿派人數遠
                 |
星宿派人數遠較 -----
                 |
                 -> 宿派人數遠較


                 -> 過不多時，賈
                 |
過不多時，賈老 -----
                 |
                 -> 不多時，賈老
```                   

到此為此，我們已經使用 `tf.data` 建立一個可以拿來給模型使用的資料集了。

TensorFlow 2.0 預設就是 [Eager Execution](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/eager.ipynb)，因此你不再需要使用老朋友 `tf.Session()` 或是 `tf.placeholder` 就能非常直覺地存取數據：

In [152]:
# print 是用來幫你理解 tf.data.Dataset
# 的內容，實際上存取資料集非常簡單
# 現在先關注下面的 print 結果
for b_inp, b_tar in ds.take(1):
    d = tokenizer.index_word
    print("起始句子的 batch：")
    print(b_inp, "\n")
    print("目標句子的 batch：")
    print(b_tar, "\n")
    print("-" * 20, "\n")
    
    print("第一個起始句子的索引序列：")
    first_i = b_inp.numpy()[0]
    print(first_i, "\n")
    print("第一個目標句子的索引序列：")
    first_t = b_tar.numpy()[0]
    print(first_t, "\n")
    print("-" * 20, "\n")
    
    print("第一個起始句子的文本序列：")
    print([d[i] for i in first_i])
    print()
    print("第一個目標句子的文本序列：")
    print([d[i] for i in first_t])

起始句子的 batch：
tf.Tensor(
[[  19   31   84 ...  111    2    3]
 [1454  120    2 ...   10   12    7]
 [ 313  516  136 ...   41   31   17]
 ...
 [   9    5  396 ...    3   24   18]
 [1063  508  547 ...    3    3  153]
 [  23   16  107 ...  641    1   14]], shape=(128, 10), dtype=int32) 

目標句子的 batch：
tf.Tensor(
[[  31   84 1626 ...    2    3    3]
 [ 120    2  114 ...   12    7   94]
 [ 516  136  937 ...   31   17    1]
 ...
 [   5  396   83 ...   24   18  301]
 [ 508  547  146 ...    3  153   62]
 [  16  107  260 ...    1   14   53]], shape=(128, 10), dtype=int32) 

-------------------- 

第一個起始句子的索引序列：
[  19   31   84 1626   18  238  125  111    2    3] 

第一個目標句子的索引序列：
[  31   84 1626   18  238  125  111    2    3    3] 

-------------------- 

第一個起始句子的文本序列：
['來', '到', '兩', '扇', '大', '石', '門', '前', '。', '\n'] 

第一個目標句子的文本序列：
['到', '兩', '扇', '大', '石', '門', '前', '。', '\n', '\n']


為了讓你理解資料集回傳的內容，上面用了不少 `print`。但事實上資料集 `ds` 負責的就是每次吐出 128 筆數據的 Tensor，而每筆數據裡頭包含了長度為 10 的輸入・輸出的數字序列（因此維度為 `(128, 10)`）。

去除 `print`，你要從資料集 `ds` 取得一個 batch 非常地簡單：

In [ ]:
for b_inp, b_tar in ds.take(1):
    # 蒙多想去哪就去哪
    # 想怎麼存取 b_iup, b_tar 都可以

### 4. 定義能解決問題的函式集

呼！我們花了不少時間在建構資料集，是時候捲起袖子將這些資料丟入模型了！

!image
- lstm-text-generation/How_to_Roll_Up_Sleeves_01.jpg

回想資料集內容，你現在應該已經很清楚我們想要模型解決的問題是什麼了：丟入一個數字序列，模型要能產生包含下個時間點的數字序列，越相似越好。

如同我們在 [AI 如何找出你的喵](https://demo.leemeng.tw/)裡頭說過的：

!quote
- 任何類型的神經網路本質上都是一個映射函數。它們會在內部進行一連串特定的數據轉換步驟，想辦法將給定的輸入數據轉換成指定的輸出形式。 

我們現在要做的就是定義一個神經網路架構，讓這個神經網路（或稱函式）幫我們把輸入的數字序列轉換成對應的輸出序列。

我們期待這個模型具有「記憶」，能考慮以前看過的所有歷史資訊，進而產生最有可能的下個中文字。

!image
- nlp-kaggle-intro/rnn-animate.gif
- 循環神經網路非常適合處理具有順序關係的數據

而在[自然語言處理與深度學習入門指南](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html)我們就已經談過，循環神經網路中的 LSTM 模型非常適合拿來做這件事情。

因此雖然理論上你可以用任意架構的神經網路如基本的前饋神經網路來解決這個問題，使用 LSTM 是一個相對較適合的選擇。在 TensorFlow 裡頭，使用 [Keras API](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras) 建立一個神經網路就像在疊疊樂，是一件很輕鬆的事情：

In [5]:
#ignore
BATCH_SIZE = 128

In [ ]:
# 超參數
EMBEDDING_DIM = 512
RNN_UNITS = 1024

# 使用 keras 建立一個非常簡單的 LSTM 模型
model = tf.keras.Sequential()

# 詞嵌入層
model.add(
    tf.keras.layers.Embedding(
        input_dim=num_words, 
        output_dim=EMBEDDING_DIM,
        batch_input_shape=[
            BATCH_SIZE, None]
))

# LSTM 層
model.add(
    tf.keras.layers.LSTM(
    units=RNN_UNITS, 
    return_sequences=True, 
    stateful=True, 
    recurrent_initializer='glorot_uniform'
))

# 全連接層
model.add(
    tf.keras.layers.Dense(
        num_words))

model.summary()

!image
- lstm-text-generation/model_summary.jpg

這邊我們建立了一個由詞嵌入層、LSTM 層以及全連接層組成的簡單 LSTM 模型。此模型一次吃 128 筆輸入數據，並產出 128 筆 4330 維度的 Tensor。

如果你還記得，4330 是天龍八部裡頭所有出現過的中文字的數目。模型輸出的每個維度值，都可以被視為是跟某個詞出現機率成正比的值。

!image
- lstm-text-generation/antoine-dautry-428776-unsplash.jpg

值得一提的是，儘管這個神經網路（或稱映射函數）看起來非常有希望能解決我們的序列生成問題，我們並不僅是建立了 1 個幫我們將輸入序列對應到目標序列的映射函數而已。事實上，我們用 `tf.keras` 定義了一個有接近 1,300 萬參數的函式**集合**（Function set）。

這跟你看到一個資料集裡頭的特徵 `x` 跟目標值 `y` 成直線，然後想用 `a * x + b = y` 形式去 fit `y` 的道理是一樣的：你相信 `a * x + b = y` 形式的映射函數能幫你把輸入 `x` 有效地對應到目標 `y`，你只是不知道最佳的參數組合 `(a, b)` 是多少罷了；同理，很多研究結果顯示文中定義的 RNN 與 LSTM 能幫我們很好的 model 序列數據，我們只是還不知道最適合天龍八部的參數組合是什麼而已。

!image
- nlp-kaggle-intro/backpropagation-example.gif
- 深度學習中我們常使用梯度下降與反向傳播來從函數集合中找出最好的函數（某個特定參數組合的神經網路架構）

參數 `a` 以及 `b` 有無限多種組合，而每一組 `a` 與 `b` 的組合都對應到一個實際的「函數」，也都能幫你把 `x` 乘上 `a` 倍再加上 `b` 後想辦法去接近目標值 `y`，只是每個函數的表現不一而已。而把這些所有可能的函數放在一起，就是所謂的函數集合。

只不過針對 `a * x + b = y` 這個簡單例子，我們可以直接用線性代數從整個函式集合裡頭瞬間找出最佳的函數 `f`（即某個參數的組合）；而我們需要透過[梯度下降（Gradient Descent）](https://zh.wikipedia.org/zh-tw/%E6%A2%AF%E5%BA%A6%E4%B8%8B%E9%99%8D%E6%B3%95)與[反向傳播算法（Backpropagation）](https://zh.wikipedia.org/wiki/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD%E7%AE%97%E6%B3%95)來幫我們在茫茫無垠的函式集合（某種神經網路架構如本文定義的 `model` ）裡頭找出一個好的神經網路（某個 1,300 萬個參數的組合）。

### 5. 定義評量函式好壞的指標



值得一提的是，透過定義這樣的輸入輸出關係，我們將一個沒有正確解答的任務轉換成監督式學習的任務了。

### 6. 訓練並選擇出最好的函式

### 7. 將函式 / 模型拿來做預測

## 結語

人跟機器的差別：我們理解世界觀，機器理解統計關係
我看十遍也寫不出類似的文章，機器看十遍無法理解天龍八部裡頭眾生的愛恨情仇。


向金庸致敬

In [ ]:
#ignore
以下為 tfjs app 部署用的 js。記得每次 `yarn build` 以後都要使用最新的 js 

<script src="{static}tfjs-apps/lstm-text-generation/dist/lstm-text-generation.03657dc5.js"></script>